In [6]:
import requests
from dotenv import load_dotenv
import os
import pandas as pd

# Load environment variables
load_dotenv()

# Retrieve API key from environment variables
api_key = os.getenv("CENSUS_API_KEY")

# Base URL for the Census API
base_url = "https://api.census.gov/data"

# Specify the year, dataset, and variables you are interested in
year = "2020"
dataset = "acs/acs5"
variables = "NAME,B01001_001E"  # Example variable for total population

county2fips = {
    "Barrow": "13013",
    "Bartow": "13015",
    "Butts": "13035",
    "Carroll": "13045",
    "Cherokee": "13057",
    "Clayton": "13063",
    "Cobb": "13067",
    "Coweta": "13077",
    "Dawson": "13085",
    "DeKalb": "13089",
    "Douglas": "13097",
    "Fayette": "13113",
    "Forsyth": "13117",
    "Fulton": "13121",
    "Gwinnett": "13135",
    "Haralson": "13143",
    "Heard": "13149",
    "Henry": "13151",
    "Jasper": "13159",
    "Lamar": "13171",
    "Meriwether": "13199",
    "Morgan": "13211",
    "Newton": "13217",
    "Paulding": "13223",
    "Pickens": "13227",
    "Pike": "13231",
    "Rockdale": "13247",
    "Spalding": "13255",
}
df = pd.DataFrame()
for county, fips in county2fips.items():
    st_fips = fips[:2]
    cty_fips = fips[2:]
    url = f"https://api.census.gov/data/2022/acs/acs5?get=NAME,B01001_001E&for=block group:*&in=state:{st_fips}+county:{cty_fips}&key={api_key}"
    resp = requests.get(url)

    data = resp.json()
    df = pd.concat([df, pd.DataFrame(data=data[1:], columns=data[0])])


In [7]:
import geopandas as gpd

# Specify the output directory for TIGER files
output_dir = "data/tiger_files/"

# Create a GeoDataFrame from the DataFrame containing block group data
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df["longitude"], df["latitude"]))

# Download TIGER files for each block group
for index, row in gdf.iterrows():
    state = row["state"]
    county = row["county"]
    tract = row["tract"]
    block_group = row["block group"]

    # Construct the URL for downloading the TIGER file
    url = f"https://www2.census.gov/geo/tiger/TIGER{year}/BG/tl_{year}_{state}{county}{tract}{block_group}.zip"

    # Download the TIGER file
    gpd.read_file(url).to_file(
        output_dir + f"tl_{year}_{state}{county}{tract}{block_group}.shp"
    )


ModuleNotFoundError: No module named 'geopandas'

In [5]:
df.to_csv("data/atlanta_metro_block.csv", index=False)
